In [6]:
# %run imports_and_functions.ipynb
# df = pd.read_csv('SeattleBusinesses.csv', sep=';', compression='gzip')
# df = make_Xmatrix(df)

# df['alias_cats'].map((lambda x : str(len(x))+x)).value_counts()
# # All functions checked and do work

In [3]:
def make_Xmatrix(df) :
    dropduplicate_yelp_entries(df)
    df['price'] = df['price'].map(convertpricecolumn)
    rowstodrop = df[df['latitude'].isnull() | df['longitude'].isnull()].index
    df.drop(rowstodrop, axis=0, inplace=True)

    df['lat_index'] = round(df['latitude'], 3)
    df['long_index'] = round(df['longitude'], 4)
    df = df.set_index(['lat_index','long_index'], drop=False).sort_index()
    df = df.sort_index(level = [0,1])
    df['alias_cats'] = df.apply(categories_to_alias2, axis=1)
    return df

def makecorpus(df, lat, long) :
    distances = calcDistancesV2(df, lat, long)
    distances = pd.DataFrame(data=distances, index=distances.index, columns=['distances'])
    distances = distances.join(df['alias_cats']) #['alias'] = df.loc[distances.index, 'alias']
#     print(distances.shape)
    temp = distances.apply(addproximity, axis=1)
    return ' '.join(temp)


lat1000m = -115.161605-(-115.172713)

def addproximity(dfrow) :
    w = ('near' if (dfrow.loc['distances'] <= 500) else 'far')
    temp = dfrow.loc['alias_cats'].split(' ')
    temp = list(map((lambda x : (x + '_' + w)), temp))
    return ' '.join(temp)

def calcDistancesV2(multiindexdf, lat, long) :
    minlat = lat - lat1000m
    maxlat = lat + lat1000m
    minlong = long - lat1000m
    maxlong = long + lat1000m
#     multiindexdf = multiindexdf.sort_index(level = [0,1])
    idx = pd.IndexSlice
    df = multiindexdf.loc[idx[minlat:maxlat, minlong:maxlong], ['latitude','longitude']]
    distances = pd.Series(index=df.index)
    for index in df.index :
        try :
            distances[index] = calcDist(lat, long, df.loc[index,'latitude'].values[0], 
                                    df.loc[index,'longitude'].values[0])
        except : # catch very rare attribute error to fill in rest of rows
            distances[index] = 10000 # if calcDist gives error, don't include this business
    distances = distances[distances<1000]
    return distances


cat_alias_dict = json.load(open('cat_alias_dict.json','r'))

def categories_to_alias2(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    rawlis = list(set(catstring.split(', ')))
    lis = []
    for item in rawlis :
        if 'alias' in item :
            alias = item.split(':')[1].strip().strip("'")
            lis += [alias]
#     print(lis)
    lis = list(map(get_alias2,lis))
    lis = [item for item in lis if item != '_other']
    lis += ['_' + dfrow['main_category']]
    # some new categories are not in category dictionary, and each row already has main category
    # so am taking out all instances where business category could not be found
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias2(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat_alias_dict[cat])])
    except : alias = '_other'
    return alias

def categories_to_alias(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    lis = list(set(catstring.split(', ')))
    lis = map(get_alias,lis)
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat.lower())])
    except : alias = '_other'
    return alias

## Function to map titles to macro-categories if not in top 150

dict_2A = json.load(open('dict_2A.json','r'))
dict_2B = json.load(open('dict_2B.json','r'))

def map_category(category):
    if category not in top_148:
        if category in dict_2B.keys():
            title = dict_2B[category]
        else:
            title = category
        if title in dict_2A.keys():
            title_2 = dict_2A[title]
            return(title_2)
        else:
            return(title)
    else:
        return(category)

def dropduplicate_yelp_entries(df) :
    dups = df[df[['latitude','longitude','alias']].duplicated(keep=False)].sort_values('latitude')
    mask1 = dups[['latitude','longitude','alias']].duplicated(keep='first')
    mask2 = dups[['latitude','longitude','alias']].duplicated(keep=False)
    mask = mask2 & ~mask1
#     print(dups)
    for index in dups[mask].index :
        dupset = dups[(dups['alias']==dups.loc[index,'alias']) & (dups['longitude']==dups.loc[index,'longitude'])].sort_values(
            'review_count', ascending=False)
        dropindex = dupset.iloc[1:,0].index
        df.drop(dropindex, axis=0, inplace=True)
        
def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0
    
def convertpricecolumn(pricesymbol) :
    try : num = len(pricesymbol)
    except : num = 0
    return num

def get_main_category(catstring) :
    string = ''
    try :
        for cat in main_categories :
            if cat in catstring : string = string + cat + ', '
    except : pass
    return string[0:(len(string)-2)]